### Pull Backcasted Factor Risk

Define your positions and risk model:

In [ ]:
import datetime as dt
from gs_quant.datetime import business_day_offset

from gs_quant.markets.portfolio import Portfolio
from gs_quant.markets.portfolio_manager import PortfolioManager
from gs_quant.markets.position_set import PositionSet
from gs_quant.markets.report import FactorRiskReport
from gs_quant.session import GsSession, Environment

GsSession.use(Environment.PROD)

positions = [
    {
        'identifier': 'AAPL UW',
        'quantity': 25
    }, {
        'identifier': 'GS UN',
        'quantity': 50
    }, {
        'identifier': 'FB UW',
        'quantity': 25
    }, {
        'identifier': 'AMZN UN',
        'quantity': 50
    }, {
        'identifier': 'MSFT UW',
        'quantity': 25
    }, {
        'identifier': 'AZN UW',
        'quantity': 50
    }
]
risk_model_id = 'AXUS4M'

print('Positions and risk model ID saved.')

Create a portfolio with those positions held on the last previous business day:

In [ ]:
portfolio = Portfolio(name='My Backcasted Portfolio')
portfolio.save()
pm = PortfolioManager(portfolio.id)
pm.update_positions([PositionSet.from_dicts(date=business_day_offset(dt.date.today(), -1, roll='forward'),
                                            positions=positions)])
pm.schedule_reports(backcast=True)
risk_report = FactorRiskReport(risk_model_id='AXUS4M', fx_hedged=True)
risk_report.set_position_source(portfolio.id)
risk_report.save()

print(f'Portfolio created with ID "{portfolio.id}".')

Run risk calculations backcasted one year:

In [ ]:
risk_results = risk_report.run(backcast=True, is_async=False)
risk_results = risk_results[risk_results['factorCategory'] == 'Aggregations']
factor_exposures = risk_results.filter(items=['date', 'factor', 'exposure']).pivot(index='date', columns='factor', values='exposure')
factor_pnl= risk_results.filter(items=['date', 'factor', 'pnl']).pivot(index='date', columns='factor', values='pnl')
factor_exposures.plot(title='Factor Exposures')
factor_pnl.cumsum().plot(title='Factor PnL')


print(f'Compare to your portfolio UI here: https://marquee.gs.com/s/portfolios/{portfolio.id}/attribution')